In [ ]:
!pip install -q streamlit
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.6 MB/s eta 0:00:00


In [ ]:
!ngrok authtoken 2XJcMWVd8A5gZbLT1Mbd8f5q8id_2QJzWvWHVpCEWDDA8XUmD

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!ngrok

In [ ]:
%%writefile app.py
import streamlit as st
import nltk
import string
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

nltk.download('punkt')
nltk.download('wordnet')

lemmer = nltk.stem.WordNetLemmatizer()

GREET_INPUTS = ['hello', 'hi', 'hey']
GREET_RESPONSES = ['hi', 'hey', "what's up", 'I am glad! You are talking to me']

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)


def convert_to_lower(text):
    return text.lower()

def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]


def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

def greet(sentence):
    for word in sentence.split():
        if word.lower() in GREET_INPUTS:
            return random.choice(GREET_RESPONSES)

def bot_response(user_response, sent_tokens):
    robo1_response = ''
    TfidVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    tfidf = TfidVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    print(vals.shape)
    idx = vals.argsort()[0][-2]
    print(idx)
    flat = vals.flatten()
    flat.sort()
    print(flat.shape)
    req_tfidf = flat[-2]
    print(req_tfidf)

    if(req_tfidf==0):
        robo1_response = robo1_response + "I am sorry! I don't understand you"
        return robo1_response

    else:
        robo1_response = robo1_response + sent_tokens[idx]
        return robo1_response

def main():

#Creating chat interface
    favicon_url = 'https://th.bing.com/th/id/OIP.4RnNLKNxFID9JiAfMeWf8gHaHa?rs=1&pid=ImgDetMain'
    st.set_page_config(page_title='Basic ChatBot', page_icon=favicon_url)

    st.header('Your Friendly Chatbot')
    st.write('I am a conversational bot, ready to answer your query! To end the chat type "exit".')


    with st.sidebar:
        st.subheader('Your Documents')
        uploaded_file = st.file_uploader('Upload your files here and press "Process"', type="txt")


    if uploaded_file is not None:
        file_contents = uploaded_file.getvalue().decode("utf-8")
        raw_doc = convert_to_lower(file_contents)
        sent_tokens = nltk.sent_tokenize(raw_doc)
        word_tokens = nltk.word_tokenize(raw_doc)

#Initializing session state
    if 'message' not in st.session_state:
        st.session_state.message = []


    for message in st.session_state.message:
        with st.chat_message(message['role']):
            st.markdown(message['content'])

#USER RESPONSE
    user_question = st.chat_input('Enter your query here')

    if user_question:
        with st.chat_message('user'):
            st.markdown(user_question)
        st.session_state.message.append({'role': "user", "content": user_question})


#BOT RESPONSE
    if user_question:

        response = ''
        if(user_question!='exit'):
            if(user_question == 'thanks' or user_question == 'thank you'):
                with st.chat_message('assistant'):
                    response = response + "You are Welcome"
                    st.markdown(response)
                st.session_state.message.append({"role": "assistant", "content": response})

            else:
                if(greet(user_question)!=None):
                    with st.chat_message('assistant'):
                        response = response + greet(user_question)
                        st.markdown(response)
                    st.session_state.message.append({"role": "assistant", "content": response})

                else:
                    sent_tokens.append(user_question)
                    word_tokens = word_tokens + nltk.word_tokenize(user_question)
                    final_words = list(set(word_tokens))
                    with st.chat_message('assistant'):
                        response = response + bot_response(user_question, sent_tokens)
                        st.markdown(response)
                    sent_tokens.remove(user_question)
                    st.session_state.message.append({"role": "assistant", "content": response})


        else:
            with st.chat_message('assistant'):
                response = response + 'GoodBye!'
                st.markdown(response)
            st.session_state.message.append({"role": "assistant", "content": response})
            st.session_state.message = []

        #st.session_state.message.append({"role": "assistant", "content": response})
        #st.session_state.message = []



if __name__ == '__main__':
    main()

Overwriting app.py


In [ ]:
!streamlit run app.py &>/dev/null&

In [ ]:
!pgrep streamlit

829


In [ ]:
from pyngrok import ngrok
# Setup a tunnel to the streamlit port 8501
public_url = ngrok.connect(8501)
public_url

<NgrokTunnel: "https://0145-34-80-52-248.ngrok-free.app" -> "http://localhost:8501">